### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, on=["school_name"], how="outer")


## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [33]:
#Calculate the total number of unique schools 
Totalno_school = len(school_data_complete["school_name"].unique())

#Calculate the total number of students
Totalno_students = len(school_data_complete["student_name"])

#Calculate the total budget
Total_budget = school_data["budget"].sum()

#Calculate the average (mean) maths score
Average_mathscore = school_data_complete["maths_score"].mean()

#Calculate the average (mean) reading score
Average_readingscore = school_data_complete["reading_score"].mean()

#calculate the percentage of students who passed maths
percentage_studentpassmath = (school_data_complete["maths_score"]>49).sum() / Totalno_students * 100

#Calculate the percentage of students who passed reading
percentage_studentreading = (school_data_complete["reading_score"]>49).sum() / Totalno_students * 100

#calculate the percentage of students that passed both maths and reading 
percentage_studentpassboth = school_data_complete.loc[(school_data_complete["maths_score"] > 49) & (
    school_data_complete["reading_score"] > 49), :]
percentage_studentpassboth_percentage = len(percentage_studentpassboth["student_name"]) / Totalno_students * 100

#Create a new DataFrame for the above calculations
area_summary = pd.DataFrame({"Total Schools" : [Totalno_school],  
                        "Total Students" : [Totalno_students],
                        "Total Budget" : Total_budget,
                        "Average Maths Score" : Average_mathscore,
                        "Average Reading Score" : Average_readingscore, 
                        "% Passing Maths" : percentage_studentpassmath,
                        "% Pass Reading" : percentage_studentreading, 
                        "% Overall Passing" : percentage_studentpassboth_percentage})
area_summary["Total Budget"] = area_summary["Total Budget"].astype(float).map("${:,.2f}".format)

area_summary
 


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Pass Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [34]:

#Use the code provided to calculate the per capita spending
school_data ["Per Student Budget"] =school_data["budget"] / school_data["size"]

#Rename the column title for the data
school_data = school_data.rename(
    columns={"school_name":"school_name",
             "type":"School Type",
             "size":"Total Students",
             "budget":"Total School Budget", 
             "Per Student Budget":"Per Student Budget"}
) 


#Grouping the data according to the school name 
student_data2 = student_data.groupby(["school_name"])
student_data2df = student_data2.sum()



school_student_complete2 = pd.merge(student_data2df, school_data, on=["school_name"], how="outer")

#Calculate the average test scores for Maths and Reading
school_student_complete2["Average Maths Score"] = school_student_complete2["maths_score"] / school_student_complete2["Total Students"]
school_student_complete2["Average Reading Score"] = school_student_complete2["reading_score"] / school_student_complete2["Total Students"]


#calculate the schools that passed both maths and reading with scores of 50 or higher
pass_student_data = school_data_complete.loc[(school_data_complete["reading_score"] > 49) & 
                                     (school_data_complete["maths_score"] > 49)]

pass_student_data ["passboth"] = 1


student_data3 = pass_student_data.groupby(["school_name"])


student_data3df = student_data3.sum()


school_student_complete3 = pd.merge(student_data3df, school_student_complete2, on=["school_name"], how="outer")
school_student_complete3["% Overall Passing"] = school_student_complete3["passboth"] / school_student_complete3["Total Students"] *100


#Calculate the number of schools with reading scores of 50 or higher 
passread_student_data = school_data_complete.loc[school_data_complete["reading_score"] > 49]

passread_student_data ["passread"] = 1


student_data4 = passread_student_data.groupby(["school_name"])



student_data4df = student_data4.sum()


school_student_complete4 = pd.merge(student_data4df, school_student_complete3, on=["school_name"], how="outer")
school_student_complete4["% Passing Reading"] = school_student_complete4["passread"] / school_student_complete4["Total Students"] *100

#Calculate the number of schools with maths scores of 50 or higher
passmaths_student_data = school_data_complete.loc[school_data_complete["maths_score"] > 49]

passmaths_student_data ["passmaths"] = 1


student_data5 = passmaths_student_data.groupby(["school_name"])


student_data5df = student_data5.sum()



school_student_complete5 = pd.merge(student_data5df, school_student_complete4, on=["school_name"], how="outer")
school_student_complete5["% Passing Maths"] = school_student_complete5["passmaths"] / school_student_complete5["Total Students"] *100


#Organised data accordingly 
organised_data = school_student_complete5 [["school_name", "School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Maths Score",
                   "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]

#Create a new DataFrame for the above calculations 
per_school_summary = organised_data.rename(columns = {
    "school_name" : "School Name", "School Type" : "School Type", "Total Students" : "Total Students", 
    "Total School Budget" : "Total School Budget", "Per Student Budget" : "Per Student Budget", 
    "Average Maths Score" : "Average Maths Score", "Average Reading Score" : "Average Reading Score", 
    "% Passing Maths" : "% Passing Maths", "% Passing Reading" : "% Passing Reading", "% Overall Passing" : "% Overall Passing"

})


per_school_summary

C:\Users\yijia\AppData\Local\Temp\ipykernel_10004\2432255080.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  student_data2df = student_data2.sum()
C:\Users\yijia\AppData\Local\Temp\ipykernel_10004\2432255080.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pass_student_data ["passboth"] = 1
C:\Users\yijia\AppData\Local\Temp\ipykernel_10004\2432255080.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
1,Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
2,Figueroa High School,Government,2949,1884411,639.0,68.698542,69.077993,81.654798,82.807731,67.650051
3,Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880
4,Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
5,Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
6,Holden High School,Independent,427,248087,581.0,72.583138,71.660422,89.929742,88.524590,78.922717
7,Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
8,Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
9,Pena High School,Independent,962,585858,609.0,72.088358,71.613306,91.683992,86.590437,79.209979


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [35]:
#Show Top 5 Performing Schools by % Overall passing
top_schools = per_school_summary.sort_values("% Overall Passing", ascending = False )

#Save the results to a DataFrame
top_schools = top_schools.reset_index(drop=True)

#Display the first 5 rows
top_schools.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
1,Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
2,Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
3,Wright High School,Independent,1800,1049400,583.0,72.047222,70.969444,91.777778,86.666667,79.722222
4,Rodriguez High School,Government,3999,2547363,637.0,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [36]:
#Show 5 Worst Performing Schools by % Overall passing
bottom_schools= per_school_summary.sort_values("% Overall Passing", ascending = True )

#Save the results to a DataFrame called
bottom_schools = bottom_schools.reset_index(drop=True)

#Display the first 5 rows
bottom_schools.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
1,Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
2,Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
3,Wilson High School,Independent,2283,1319574,578.0,69.170828,68.876916,82.785808,81.296540,67.455103
4,Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [37]:
#Sort out data when == to each year level
Year_9 = school_data_complete[(school_data_complete["year"] == 9)]
Year_10 = school_data_complete[(school_data_complete["year"] == 10)]
Year_11 = school_data_complete[(school_data_complete["year"] == 11)]
Year_12 = school_data_complete[(school_data_complete["year"] == 12)]

#Calculate the mean for each year level in maths
Year_9_scores = Year_9.groupby(["school_name"])["maths_score"].mean(numeric_only=True)
Year_10_scores = Year_10.groupby(["school_name"])["maths_score"].mean(numeric_only=True)
Year_11_scores = Year_11.groupby(["school_name"])["maths_score"].mean(numeric_only=True)
Year_12_scores = Year_12.groupby(["school_name"])["maths_score"].mean(numeric_only=True)

#Create a data frame for the data summary
scores_summary = pd.DataFrame({"Year 9" : Year_9_scores, 
"Year 10" : Year_10_scores, "Year 11" : Year_11_scores,
"Year 12": Year_12_scores})

scores_summary = scores_summary[["Year 9", "Year 10", "Year 11", "Year 12"]]


scores_summary


,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


## Reading Score by Year

* Perform the same operations as above for reading scores

In [38]:
#Sort out data when == to each year level
Year_9_R = school_data_complete[(school_data_complete["year"] == 9)]
Year_10_R = school_data_complete[(school_data_complete["year"] == 10)]
Year_11_R= school_data_complete[(school_data_complete["year"] == 11)]
Year_12_R= school_data_complete[(school_data_complete["year"] == 12)]

#Calculate the mean for each year level in reading
Year_9_Rscores = Year_9_R.groupby(["school_name"])["reading_score"].mean(numeric_only=True)
Year_10_Rscores = Year_10_R.groupby(["school_name"])["reading_score"].mean(numeric_only=True)
Year_11_Rscores = Year_11_R.groupby(["school_name"])["reading_score"].mean(numeric_only=True)
Year_12_Rscores = Year_12_R.groupby(["school_name"])["reading_score"].mean(numeric_only=True)

#Create a data frame for the data summary
Rscores_summary = pd.DataFrame({"Year 9" : Year_9_Rscores, 
"Year 10" : Year_10_Rscores, "Year 11" : Year_11_Rscores,
"Year 12": Year_12_Rscores})

Rscores_summary = Rscores_summary[["Year 9", "Year 10", "Year 11", "Year 12"]]


Rscores_summary

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [39]:
#Create bins by the spending range
bins = [0, 585, 630, 645, 680]

#Create labels for the spending range
groupname = ["<$585", "$585-630", "$630-645", "$645-680"]

#Slice the data and place it into bins according to the spending 
per_school_summary["Spending Ranges (Per Student)"]=pd.cut(per_school_summary["Total School Budget"]/per_school_summary["Total Students"], 
                                                           bins, labels=groupname)


#Calculate the mean of scores vs spending per student
spending_math_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_passing_reading = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

#Create a data frame for the summary
spending_summary = pd.DataFrame({"Average Maths Score": spending_math_scores,
                                 "Average Reading Score": spending_reading_scores,
                                 "% Passing Maths": spending_passing_math,
                                 "% Passing Reading": spending_passing_reading,
                                 "% Overall Passing": overall_passing_spending})

spending_summary = spending_summary[["Average Maths Score",
                                     "Average Reading Score",
                                     "% Passing Maths",
                                     "% Passing Reading",
                                     "% Overall Passing"]]
spending_summary


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


## Scores by School Size

* Perform the same operations as above, based on school size.

In [40]:
#Create bins for the range of school size
bins2 = [0, 1000, 2000, 5000]

#Create labels for the school size
groupname2 = ["Small (<1000)", "Medium (1000-2000)", "Large (20000-5000)"]


#Slice the data and place it into bins according number of students
per_school_summary["School Size"]=pd.cut(per_school_summary["Total Students"], 
                                                           bins2, labels=groupname2)


#Calculate the mean of scores vs school size

size_math_scores = per_school_summary.groupby(["School Size"])["Average Maths Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Maths"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_size = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

#Create a data frame for the summary
schoolsize_summary = pd.DataFrame({"Average Maths Score": size_math_scores,
                                 "Average Reading Score": size_reading_scores,
                                 "% Passing Maths": size_passing_math,
                                 "% Passing Reading": size_passing_reading,
                                 "% Overall Passing": overall_passing_size})

schoolsize_summary = schoolsize_summary[["Average Maths Score",
                                     "Average Reading Score",
                                     "% Passing Maths",
                                     "% Passing Reading",
                                     "% Overall Passing"]]
schoolsize_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (20000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

In [41]:

#Calculate the mean of scores vs school type

type_math_scores = per_school_summary.groupby(["School Type"])["Average Maths Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_math = per_school_summary.groupby(["School Type"])["% Passing Maths"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
overall_passing_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

#Create a data frame for the summary
schooltype_summary = pd.DataFrame({"Average Maths Score": type_math_scores,
                                 "Average Reading Score": type_reading_scores,
                                 "% Passing Maths": type_passing_math,
                                 "% Passing Reading": type_passing_reading,
                                 "% Overall Passing": overall_passing_type})

schooltype_summary = schooltype_summary[["Average Maths Score",
                                     "Average Reading Score",
                                     "% Passing Maths",
                                     "% Passing Reading",
                                     "% Overall Passing"]]
schooltype_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
